In [1]:
!pip install simpletransformers

In [2]:
import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score

from seqeval.metrics import classification_report, accuracy_score, f1_score
from tqdm import tqdm,trange

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dataset_path = '/content/drive/MyDrive/Colab Notebooks/ner-drone/ner_drone_train_2.csv'
# data_test_path = '/content/drive/MyDrive/Colab Notebooks/ner-drone/ner_drone_test_2.csv'
dataset = pd.read_csv(dataset_path, encoding= 'unicode_escape')
# dataset_test = pd.read_csv(data_test_path, encoding= 'unicode_escape')
dataset

,sentence_idx,word,tag
0,1,Taking,B-ACT
1,1,Off,I-ACT
2,2,Taking,B-ACT
3,2,Off,I-ACT
4,3,Taking,B-ACT
...,...,...,...
3673,672,will,O
3674,672,be,O
3675,672,disabled,O
3676,672,in,O


In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [6]:
dataset.rename(columns={"sentence_idx":"sentence_id","word":"words","tag":"labels"}, inplace =True)
# dataset_test.rename(columns={"sentence_idx":"sentence_id","word":"words","tag":"labels"}, inplace =True)
dataset

,sentence_id,words,labels
0,1,Taking,B-ACT
1,1,Off,I-ACT
2,2,Taking,B-ACT
3,2,Off,I-ACT
4,3,Taking,B-ACT
...,...,...,...
3673,672,will,O
3674,672,be,O
3675,672,disabled,O
3676,672,in,O


In [7]:
from simpletransformers.ner import NERModel, NERArgs

In [8]:
label = dataset["labels"].unique().tolist()
label

['B-ACT',
 'I-ACT',
 'B-ISS',
 'I-ISS',
 'O',
 'B-CMP',
 'B-FNC',
 'I-FNC',
 'I-CMP',
 'B-STE',
 'I-STE']

In [9]:
args = NERArgs()
args.num_train_epochs = 3
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

In [10]:
model = NERModel('bert', 'bert-base-cased', labels=label, args =args, use_cuda=False)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [13]:
train_size = 0.7
dataset_train = dataset.sample(frac=train_size,random_state=200)
dataset_test = dataset.drop(dataset_train.index).reset_index(drop=True)
dataset_train = dataset_train.reset_index(drop=True)

print("FULL Dataset: {}".format(dataset.shape))
print("TRAIN Dataset: {}".format(dataset_train.shape))
print("TEST Dataset: {}".format(dataset_test.shape))

FULL Dataset: (3678, 3)
TRAIN Dataset: (2575, 3)
TEST Dataset: (1103, 3)


In [14]:
model.train_model(dataset_train,eval_data = dataset_test, acc=accuracy_score)

  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/21 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/21 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/21 [00:00<?, ?it/s]

(63, 0.41505052319299135)

In [15]:
result, model_outputs, preds_list = model.eval_model(dataset_test)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/16 [00:00<?, ?it/s]

In [16]:
result

{'eval_loss': 0.14754813228501007,
 'precision': 0.9043824701195219,
 'recall': 0.8901960784313725,
 'f1_score': 0.8972332015810275}

In [17]:
preds_list

[['B-ACT'],
 ['B-ACT', 'O'],
 ['O', 'O', 'O'],
 ['O', 'O'],
 ['B-FNC', 'O'],
 ['O', 'O', 'B-CMP', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'B-CMP', 'O', 'O'],
 ['O', 'O', 'O'],
 ['I-FNC'],
 ['O', 'O'],
 ['O'],
 ['O', 'O', 'O'],
 ['B-FNC', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O'],
 ['O'],
 ['I-ACT'],
 ['I-ACT'],
 ['O'],
 ['B-ACT', 'I-ACT'],
 ['B-ACT', 'I-ACT'],
 ['I-ACT'],
 ['O', 'O', 'O', 'O'],
 ['O', 'O'],
 ['B-ACT'],
 ['O'],
 ['B-FNC'],
 ['I-ISS', 'O', 'O', 'O', 'O'],
 ['O', 'O'],
 ['O'],
 ['O', 'B-ACT', 'B-ACT', 'O'],
 ['B-ACT'],
 ['B-ACT'],
 ['O'],
 ['I-ISS', 'O'],
 ['I-CMP'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['B-ACT'],
 ['B-ACT'],
 ['O', 'O'],
 ['B-ACT'],
 ['O'],
 ['O', 'B-CMP', 'O'],
 ['O', 'O', 'O', 'O'],
 ['O', 'B-ACT'],
 ['B-ACT'],
 ['O', 'O'],
 ['O', 'O'],
 ['I-ACT'],
 ['B-CMP', 'I-CMP'],
 ['B-ACT', 'O', 'O', 'B-CMP', 'O', 'O'],
 ['O', 'O', 'O'],
 ['I-STE'],
 ['O', 'O', 'O', 'O', 'B-ACT'],
 ['O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O'],
 ['O'],
 [

In [18]:
y_test = dataset_test['labels']
y_test

0       B-ACT
1       B-ACT
2           O
3           O
4           O
        ...  
1098        O
1099        O
1100        O
1101        O
1102    B-ACT
Name: labels, Length: 1103, dtype: object

In [19]:
len(preds_list)

493

In [20]:
y_pred_list = []
for row in range(0, len(preds_list)):
  y_pred_list = np.concatenate((y_pred_list, preds_list[row]), axis=0)
y_pred_list

array(['B-ACT', 'B-ACT', 'O', ..., 'B-FNC', 'O', 'B-ACT'], dtype='<U32')

In [21]:
data_prediction = pd.DataFrame({'actual_class': y_test.to_list(), 'predicted_class': y_pred_list})
data_prediction

,actual_class,predicted_class
0,B-ACT,B-ACT
1,B-ACT,B-ACT
2,O,O
3,O,O
4,O,O
...,...,...
1098,O,O
1099,O,O
1100,O,B-FNC
1101,O,O


In [22]:
confusion_matrix = pd.crosstab(data_prediction['predicted_class'], data_prediction['actual_class'])
confusion_matrix

actual_class,B-ACT,B-CMP,B-FNC,B-ISS,B-STE,I-ACT,I-CMP,I-FNC,I-ISS,I-STE,O
predicted_class,,,,,,,,,,,
B-ACT,31,4,1,0,0,0,0,0,0,0,1
B-CMP,0,37,0,0,0,0,0,0,0,0,1
B-FNC,0,0,101,0,0,0,0,0,0,0,2
B-ISS,0,0,0,9,0,0,0,0,0,0,0
B-STE,0,0,0,0,24,0,0,0,0,1,0
I-ACT,0,0,0,0,0,9,0,0,0,0,0
I-CMP,0,0,0,0,0,0,8,0,0,0,0
I-FNC,0,0,0,0,0,0,0,5,0,0,0
I-ISS,0,0,0,0,0,0,0,0,21,1,0


In [33]:
y_test = data_prediction['actual_class'].to_list()
y_pred = data_prediction['predicted_class'].to_list()

In [35]:
from seqeval.metrics import classification_report
pd.DataFrame(classification_report([y_test], [y_pred], output_dict=True)).T

,precision,recall,f1-score,support
ACT,0.853659,1.000000,0.921053,35.0
CMP,0.976744,0.893617,0.933333,47.0
FNC,0.980952,0.962617,0.971698,107.0
ISS,0.680000,0.531250,0.596491,32.0
STE,0.750000,0.857143,0.800000,28.0
micro avg,0.898374,0.887550,0.892929,249.0
macro avg,0.848271,0.848925,0.844515,249.0
weighted avg,0.897618,0.887550,0.889811,249.0


In [24]:
prediction, model_output = model.predict(["Aircraft is returning to the Home Point Minimum RTH Altitude is 30m You can reset the RTH Altitude in Remote Controller Settings after cancelling RTH"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
prediction

[[{'Aircraft': 'B-CMP'},
  {'is': 'O'},
  {'returning': 'O'},
  {'to': 'O'},
  {'the': 'O'},
  {'Home': 'O'},
  {'Point': 'O'},
  {'Minimum': 'O'},
  {'RTH': 'B-ACT'},
  {'Altitude': 'O'},
  {'is': 'O'},
  {'30m': 'O'},
  {'You': 'O'},
  {'can': 'O'},
  {'reset': 'O'},
  {'the': 'O'},
  {'RTH': 'B-ACT'},
  {'Altitude': 'O'},
  {'in': 'O'},
  {'Remote': 'O'},
  {'Controller': 'B-CMP'},
  {'Settings': 'B-FNC'},
  {'after': 'O'},
  {'cancelling': 'O'},
  {'RTH': 'B-ACT'}]]

In [26]:
prediction2, model_output2 = model.predict(["Weak signal Avoid blocking the antennas and keep the antennas parallel to and facing toward the aircraft during flight"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
prediction2

[[{'Weak': 'O'},
  {'signal': 'O'},
  {'Avoid': 'O'},
  {'blocking': 'O'},
  {'the': 'O'},
  {'antennas': 'O'},
  {'and': 'O'},
  {'keep': 'O'},
  {'the': 'O'},
  {'antennas': 'O'},
  {'parallel': 'O'},
  {'to': 'O'},
  {'and': 'O'},
  {'facing': 'O'},
  {'toward': 'O'},
  {'the': 'O'},
  {'aircraft': 'B-CMP'},
  {'during': 'O'},
  {'flight': 'O'}]]

In [28]:
import torch
torch.save(model, '/content/drive/MyDrive/Colab Notebooks/ner-drone/simpletransformer_ner_drone(random_salah).h5')